In [5]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator

Using TensorFlow backend.


In [1]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 240/Project/Data


Spectrograms/


In [0]:
img = load_img('Spectrograms/train/concentration/S001E02_AF3.png')

In [13]:
print(type(img))
print(img.format)
print(img.mode)
print(img.size)

<class 'PIL.Image.Image'>
None
RGB
(432, 288)


In [14]:
print(type(img))
# convert to numpy array
img_array = img_to_array(img)
print(img_array.dtype)
print(img_array.shape)
# convert back to image
img_pil = array_to_img(img_array)
print(type(img))

<class 'PIL.Image.Image'>
float32
(288, 432, 3)
<class 'PIL.Image.Image'>


In [63]:
import os
import numpy as np
import pandas as pd
base_path = "Spectrograms/concentration"
data_array = []
for foldername in os.listdir(base_path):
  print(foldername)
  array = np.zeros((288,432,1))
  for filename in os.listdir(os.path.join(base_path, foldername)):
    example = foldername
    channel = filename.split('.')[0].split("_")[1]
    filepath = os.path.join(os.path.join(base_path, foldername), filename)
    #print(filename,filepath)
    img = load_img(filepath)
    img_array = img_to_array(img)
    #print(img_array.shape, array.shape)
    array = np.concatenate((array, img_array), axis=2)
    #print(img_array.shape, array.shape)
  data_array.append(np.delete(array,0,2))
#print(data_array['S025E02'].shape)
print(np.array(data_array).shape)

S001E02
S001E04
S002E04
S003E04
S005E04
S006E04
S009E04
S008E04
S007E04
S010E04
S011E04
S014E04
S012E04
S013E04
S016E04
S015E04
S020E04
S019E04
S018E04
S017E04
S023E04
S022E04
S021E04
S026E04
S025E04
S024E04
S030E04
S029E04
S028E04
S027E04
S002E02
S004E02
S003E02
S008E02
S007E02
S006E02
S005E02
S011E02
S010E02
S009E02
S013E02
S012E02
S015E02
S014E02
S018E02
S017E02
S016E02
S021E02
S020E02
S019E02
S025E02
S024E02
S023E02
S022E02
S026E02
S029E02
S028E02
S027E02
S004E04
S030E02
(60, 288, 432, 42)


In [9]:
datagen = ImageDataGenerator()
train_base_path = "Spectrograms/train"
test_base_path = "Spectrograms/test"
validation_base_path = "Spectrograms/validation"
train_generator = datagen.flow_from_directory(
        train_base_path,
        target_size=(288, 432),
        class_mode='binary')
test_generator = datagen.flow_from_directory(
        test_base_path,
        target_size=(288, 432),
        class_mode='binary')
validation_generator = datagen.flow_from_directory(
        validation_base_path,
        target_size=(288, 432),
        class_mode='binary')

Found 1344 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


In [10]:
# confirm the iterator works
batchX, batchy = train_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
batchX, batchy = validation_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(32, 288, 432, 3), min=0.000, max=255.000
Batch shape=(32, 288, 432, 3), min=0.000, max=255.000


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(288, 432, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=42, validation_data=validation_generator, validation_steps=6)

Epoch 1/10
42/42 [==============================] - 195s 5s/step - loss: 7.8170 - acc: 0.5097 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/10
42/42 [==============================] - 250s 6s/step - loss: 8.2796 - acc: 0.4807 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/10
42/42 [==============================] - 206s 5s/step - loss: 7.8051 - acc: 0.5104 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/10
42/42 [==============================] - 203s 5s/step - loss: 7.8763 - acc: 0.5060 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/10
42/42 [==============================] - 196s 5s/step - loss: 8.1610 - acc: 0.4881 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 6/10
42/42 [==============================] - 217s 5s/step - loss: 7.7814 - acc: 0.5119 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 7/10
42/42 [==============================] - 236s 6s/step - loss: 8.1373 - acc: 0.4896 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 8/10
42/42 [==============================] - 203s 5s/step - loss: 7.8170 - a

In [0]:
# evaluate model
loss = model.evaluate_generator(test_generator, steps=6)

In [0]:
print(loss)

[8.346871058146158, 0.48214285714285715]
